# Kevin Ferdinand 0358519 Capstone Code


## Import Modules and Install PySpark

In [1]:
!pip list

Package                 Version
----------------------- --------
absl-py                 1.3.0
aiohttp                 3.8.3
aiosignal               1.2.0
anyio                   3.5.0
appdirs                 1.4.4
argon2-cffi             21.3.0
argon2-cffi-bindings    21.2.0
asttokens               2.0.5
astunparse              1.6.3
async-timeout           4.0.2
attrs                   22.1.0
autovizwidget           0.20.0
Babel                   2.11.0
backcall                0.2.0
beautifulsoup4          4.12.2
bleach                  4.1.0
blinker                 1.4
Bottleneck              1.3.5
brotlipy                0.7.0
cachetools              4.2.2
certifi                 2023.5.7
cffi                    1.15.1
charset-normalizer      2.0.4
click                   8.0.4
colorama                0.4.6
comm                    0.1.2
contourpy               1.0.5
cryptography            39.0.1
cycler                  0.11.0
debugpy                 1.5.1
decorator               5

In [2]:
!pip install pyspark

In [3]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['HADOOP_HOME'] = 'C:\Program Files\hadoop\hadoop-3.2.1\bin'

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime

from matplotlib import pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType, FloatType
from pyspark.sql.functions import when, expr
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.tree import GradientBoostedTrees
from pyspark.ml.classification import GBTClassifier
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, array, lit

from pyspark.ml import PipelineModel
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import load_iris
from xgboost import plot_tree

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

## Create SparkSession and SparkContext Objects

In [7]:
spark = SparkSession.builder.master('local').appName('solution').config('spark.executor.memory', '8g').config('spark.driver.memory', '8g').getOrCreate()
sc = spark.sparkContext

In [8]:
# Jupyter Version
import os

data_dir = os.getcwd() 
data_dir = data_dir + "\\Dataset\\"
print("Current Directory:", data_dir)

Current Directory: C:\Users\kevin\Documents\CapstoneProject\CapstoneProjectTaylor\Dataset\


### Load Data as RDDs - txt to Dataframe


In [9]:
# Drive Version
#data_dir='/content/drive/MyDrive/Colab Notebooks/Capstone/Dataset/'


In [10]:
# Load Each Txt File and use spark to convert it into a Pipeline RDD
loanOriginationRDD2017 = sc.textFile(data_dir+'sample_orig_2017.txt').map(lambda x: x.split('|'))
loanOriginationRDD2018 = sc.textFile(data_dir+'sample_orig_2018.txt').map(lambda x: x.split('|'))
loanOriginationRDD2019 = sc.textFile(data_dir+'sample_orig_2019.txt').map(lambda x: x.split('|'))
loanOriginationRDD2020 = sc.textFile(data_dir+'sample_orig_2020.txt').map(lambda x: x.split('|'))

In [11]:
#loanOriginationRDD = loanOriginationRDD2017.union(loanOriginationRDD2018).union(loanOriginationRDD2019).union(loanOriginationRDD2020)
loanOriginationRDD = loanOriginationRDD2018.union(loanOriginationRDD2019).union(loanOriginationRDD2020)

In [12]:
monthlyPerformanceRDD2017 = sc.textFile(data_dir+'sample_svcg_2017.txt').map(lambda x: x.split('|'))
monthlyPerformanceRDD2018 = sc.textFile(data_dir+'sample_svcg_2018.txt').map(lambda x: x.split('|'))
monthlyPerformanceRDD2019 = sc.textFile(data_dir+'sample_svcg_2019.txt').map(lambda x: x.split('|'))
monthlyPerformanceRDD2020 = sc.textFile(data_dir+'sample_svcg_2020.txt').map(lambda x: x.split('|'))

#monthlyPerformanceRDD = monthlyPerformanceRDD2017.union(monthlyPerformanceRDD2018).union(monthlyPerformanceRDD2019).union(monthlyPerformanceRDD2020)
monthlyPerformanceRDD = monthlyPerformanceRDD2018.union(monthlyPerformanceRDD2019).union(monthlyPerformanceRDD2020)


### Check Content

In [13]:
print('There are', loanOriginationRDD.count(), 'records in the loan origination dataset.')
print('There are', monthlyPerformanceRDD.count(), 'records in the monthly performance dataset.')

There are 150000 records in the loan origination dataset.
There are 3943964 records in the monthly performance dataset.


## Convert it into DataFrame

In [14]:
# Create the Loan Origination DataFrame from the Loan Origination RDD.
# Next, assign new column names to that DataFrame.

loanOriginationDF = spark.createDataFrame(loanOriginationRDD).toDF(
    'CREDIT_SCORE', 
    'FIRST_PAYMENT_DATE',
    'FIRST_TIME_HOMEBUYER_FLAG',
    'MATURITY_DATE',
    'METROPOLITAN_STATISTICAL_AREA',
    'MORTGAGE_INSURANCE_PERCENTAGE',
    'NUMBER_OF_UNITS',
    'OCCUPANCY_STATUS',
    'ORIGINAL_COMBINED_LOAN_TO_VALUE',
    'ORIGINAL_DEBT_TO_INCOME_RATIO',
    'ORIGINAL_UPB',
    'ORIGINAL_LOAN_TO_VALUE',
    'ORIGINAL_INTEREST_RATE',
    'CHANNEL',
    'PREPAYMENT_PENALTY_MORTGAGE_FLAG',
    'AMORTIZATION_TYPE',
    'PROPERTY_STATE',
    'PROPERTY_TYPE',
    'POSTAL_CODE',
    'LOAN_SEQUENCE_NUMBER',
    'LOAN_PURPOSE',
    'ORIGINAL_LOAN_TERM',
    'NUMBER_OF_BORROWERS',
    'SELLER_NAME',
    'SERVICER_NAME',
    'SUPER_CONFORMING_FLAG',
    'PRERELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER',
    'PROGRAM_INDICATOR',
    'RELIEF_REFINANCE_INDICATOR',
    'PROPERTY_VALUATION_METHOD',
    'INTEREST_ONLY_INDICATOR')

### Extract DELINQUENT from Monthly Performance RDD

In [15]:
dictionary = {}
# Txt File states that 0 = no delinquency , 1 = have a delinquent 
# Extract these coreesponding to the Loan Sequence Number and check if there is any instance of delinquency
def fillDictionary(record):
    loanSequenceNumber = record[0]
    currentLoanDelinquencyStatus = record[3]
    delinquent = 0 if currentLoanDelinquencyStatus == '0' else 1
    if loanSequenceNumber in dictionary:
        if dictionary[loanSequenceNumber] == 0:
            dictionary[loanSequenceNumber] = delinquent
    else:
        dictionary[loanSequenceNumber] = delinquent

monthlyPerformanceData = monthlyPerformanceRDD.collect()

for record in monthlyPerformanceData:
    fillDictionary(record)
print('The dictionary contains', len(dictionary), 'key-value pairs.')

The dictionary contains 149997 key-value pairs.


### Create Delinquent DataFrame

In [16]:
delinquentDF = spark.createDataFrame(dictionary.items(), schema=['LOAN_SEQUENCE_NUMBER', 'DELINQUENT'])
delinquentDF.printSchema()

root
 |-- LOAN_SEQUENCE_NUMBER: string (nullable = true)
 |-- DELINQUENT: long (nullable = true)



Now Join them with the other file to get a pair of person data and delinquency

In [17]:
df = loanOriginationDF.join(delinquentDF, 'LOAN_SEQUENCE_NUMBER')  # inner join
#df.printSchema()

loanInDelinquencyCount = df.filter(df['DELINQUENT'] == 1).count()
loanNotInDelinquencyCount = df.filter(df['DELINQUENT'] == 0).count()
loanCount = loanInDelinquencyCount + loanNotInDelinquencyCount

print('There are', loanInDelinquencyCount, 'loans in delinquency.')
print('There are', loanNotInDelinquencyCount, 'loans not in delinquency.')
print('There are', loanCount, 'available loans in total.')

There are 14830 loans in delinquency.
There are 135167 loans not in delinquency.
There are 149997 available loans in total.


### Drop Columns that is not important

In [18]:
#print(df.select(df['AMORTIZATION_TYPE']).distinct().collect())  # one distinct value - drop the column!

columnsToDrop = [
    'FIRST_PAYMENT_DATE',
    'MATURITY_DATE',
    'MORTGAGE_INSURANCE_PERCENTAGE',
    'ORIGINAL_UPB',
    'ORIGINAL_INTEREST_RATE',
    'PREPAYMENT_PENALTY_MORTGAGE_FLAG',
    'PROPERTY_STATE',
    'LOAN_SEQUENCE_NUMBER',
    'SELLER_NAME',
    'SERVICER_NAME',
    'SUPER_CONFORMING_FLAG',
    'PRERELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER',
    'PROGRAM_INDICATOR',
    'RELIEF_REFINANCE_INDICATOR',
    'PROPERTY_VALUATION_METHOD',
    'INTEREST_ONLY_INDICATOR',
    'AMORTIZATION_TYPE',
    'ORIGINAL_DEBT_TO_INCOME_RATIO'    
]

df = df.drop(*columnsToDrop)

In [19]:
# Check Datatypes
df.printSchema()


root
 |-- CREDIT_SCORE: string (nullable = true)
 |-- FIRST_TIME_HOMEBUYER_FLAG: string (nullable = true)
 |-- METROPOLITAN_STATISTICAL_AREA: string (nullable = true)
 |-- NUMBER_OF_UNITS: string (nullable = true)
 |-- OCCUPANCY_STATUS: string (nullable = true)
 |-- ORIGINAL_COMBINED_LOAN_TO_VALUE: string (nullable = true)
 |-- ORIGINAL_LOAN_TO_VALUE: string (nullable = true)
 |-- CHANNEL: string (nullable = true)
 |-- PROPERTY_TYPE: string (nullable = true)
 |-- POSTAL_CODE: string (nullable = true)
 |-- LOAN_PURPOSE: string (nullable = true)
 |-- ORIGINAL_LOAN_TERM: string (nullable = true)
 |-- NUMBER_OF_BORROWERS: string (nullable = true)
 |-- DELINQUENT: long (nullable = true)



Change some Datatypes to Double

In [20]:
df = df.withColumn('CREDIT_SCORE', df['CREDIT_SCORE'].cast(DoubleType()))
df = df.withColumn('METROPOLITAN_STATISTICAL_AREA', df['METROPOLITAN_STATISTICAL_AREA'].cast(DoubleType()))
df = df.withColumn('NUMBER_OF_UNITS', df['NUMBER_OF_UNITS'].cast(DoubleType()))
df = df.withColumn('ORIGINAL_COMBINED_LOAN_TO_VALUE', df['ORIGINAL_COMBINED_LOAN_TO_VALUE'].cast(DoubleType()))
#df = df.withColumn('ORIGINAL_DEBT_TO_INCOME_RATIO', df['ORIGINAL_DEBT_TO_INCOME_RATIO'].cast(DoubleType()))
df = df.withColumn('ORIGINAL_LOAN_TO_VALUE', df['ORIGINAL_LOAN_TO_VALUE'].cast(DoubleType()))
df = df.withColumn('POSTAL_CODE', df['POSTAL_CODE'].cast(DoubleType()))
df = df.withColumn('ORIGINAL_LOAN_TERM', df['ORIGINAL_LOAN_TERM'].cast(DoubleType()))
df = df.withColumn('NUMBER_OF_BORROWERS', df['NUMBER_OF_BORROWERS'].cast(DoubleType()))

df.printSchema()

root
 |-- CREDIT_SCORE: double (nullable = true)
 |-- FIRST_TIME_HOMEBUYER_FLAG: string (nullable = true)
 |-- METROPOLITAN_STATISTICAL_AREA: double (nullable = true)
 |-- NUMBER_OF_UNITS: double (nullable = true)
 |-- OCCUPANCY_STATUS: string (nullable = true)
 |-- ORIGINAL_COMBINED_LOAN_TO_VALUE: double (nullable = true)
 |-- ORIGINAL_LOAN_TO_VALUE: double (nullable = true)
 |-- CHANNEL: string (nullable = true)
 |-- PROPERTY_TYPE: string (nullable = true)
 |-- POSTAL_CODE: double (nullable = true)
 |-- LOAN_PURPOSE: string (nullable = true)
 |-- ORIGINAL_LOAN_TERM: double (nullable = true)
 |-- NUMBER_OF_BORROWERS: double (nullable = true)
 |-- DELINQUENT: long (nullable = true)



Make 2 functions to help analyze categorical and continuos data

In [21]:
# Analyse categorical features.
# the Dataframe with Selected Attributes, Selected Column Name , The Figure Size, Title , Rotation
def analyseCategoricalData(df, column, fig_size=(12, 7), title=None, rot=90):
    # Print Unique Values and Missing Values 
    print("Number of unique values: {}\n".format(len(df[column].unique())))
    print("Number of missing values: {}\n".format(df[column].isnull().sum()))
    # Count the Missing Values
    df = np.round(df[column].value_counts(normalize=True, ascending=False, dropna=False) * 100, 2)
    if True in df.index.isnull():
        df.index = df.index.fillna("Missing Values") 
    print(df)
    # Plot the Figure using pyplot 
    fig = plt.figure(figsize=fig_size)
    ax = df.plot.bar()
    for p in ax.patches:
        ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))
    plt.title(title)
    plt.xticks(rotation=rot)
    plt.savefig(column + '.png', dpi=fig.dpi)
    plt.show()

def analyseContinuousData(df, column, xlabel="", figSize=(12, 7)):
    # Print Missing Values and Statistics such as Mean, std,count ....
    print("Number of missing values: {}\n".format(df[column].isnull().sum()))
    print('Descriptive statistics:\n')
    print(df[column].describe())

    # Plot the graphs Make it as density values to better represent it
    fig, ax = plt.subplots()
    df.loc[df.DELINQUENT == True, column].plot.hist(bins=50, density=True, ax=ax, alpha=0.4, label='Delinquent')
    df.loc[df.DELINQUENT == False, column].plot.hist(bins=50, density=True, ax=ax, alpha=0.4, label='Non-Delinquent')
    plt.title("Delinquent vs. Non-Delinquent")
    plt.xlabel(xlabel)
    plt.ylabel('Density Value')
    plt.legend()
    plt.savefig(column + '.png', dpi=fig.dpi)
    plt.show()

## Exploratory Data Analysis

### Explore CREDIT_SCORE

In [22]:
# A credit score is unavailable when its value is 9999.
print('The number of loan records with unavailable credit scores:', df.filter(df['CREDIT_SCORE'] == 9999).count())

The number of loan records with unavailable credit scores: 40


In [23]:
# Replace the values of unavailable credit scores with None.
df = df.withColumn('CREDIT_SCORE', when(df['CREDIT_SCORE'] != 9999, df['CREDIT_SCORE']).otherwise(None))

In [24]:
# Replace the values of unavailable credit scores with 0.
df = df.withColumn('CREDIT_SCORE', when(df['CREDIT_SCORE'].isNotNull(), df['CREDIT_SCORE']).otherwise(0.0))
print('The number of loan records with unavailable credit scores:', df.filter(df['CREDIT_SCORE'] == 0.0).count())

The number of loan records with unavailable credit scores: 40


Explore First Time Homebuyer Flag

In [25]:
# A first-time homebuyer flag is unavailable when its value is '9'.

print('The number of loan records with unavailable first-time homebuyer flags:', df.filter(df['FIRST_TIME_HOMEBUYER_FLAG'] == '9').count())

The number of loan records with unavailable first-time homebuyer flags: 0


### Explore METROPOLITAN_STATISTICAL_AREA


In [26]:
# MSA is codes that corresponds to a certain area and have some information about them
print('Number of unique MSA codes:', df.select(df['METROPOLITAN_STATISTICAL_AREA']).distinct().count())
print('Number of loan records with unavailable MSA codes:', df.filter(df['METROPOLITAN_STATISTICAL_AREA'].isNull()).count())

Number of unique MSA codes: 415
Number of loan records with unavailable MSA codes: 13731


replace null with 0 and keep non null as usuall


In [27]:
df = df.withColumn('METROPOLITAN_STATISTICAL_AREA', when(df['METROPOLITAN_STATISTICAL_AREA'].isNotNull(), df['METROPOLITAN_STATISTICAL_AREA']).otherwise(0.0))

### Explore NUMBER_OF_UNITS

In [28]:
# The number of units is unavailable when its value is 99.

print('The number of loan records with the unavailable number of units:', df.filter(df['NUMBER_OF_UNITS'] == 99).count())

The number of loan records with the unavailable number of units: 0


### Explore OCCUPANCY_STATUS

In [29]:
# An occupancy status is unavailable when its value is '9'.

print('The number of loan records with unavailable occupancy status:', df.filter(df['OCCUPANCY_STATUS'] == '9').count())

The number of loan records with unavailable occupancy status: 0


### Explore ORIGINAL_COMBINED_LOAN_TO_VALUE of property


In [30]:
# An original combined LTV is unavailable when its value is 999.

print('The number of loan records with unavailable original combined LTVs:', df.filter(df['ORIGINAL_COMBINED_LOAN_TO_VALUE'] == 999).count())

The number of loan records with unavailable original combined LTVs: 4


In [31]:
# Replace unavailable values with None for the EDA

df = df.withColumn('ORIGINAL_COMBINED_LOAN_TO_VALUE', when(df['ORIGINAL_COMBINED_LOAN_TO_VALUE'] != 999, df['ORIGINAL_COMBINED_LOAN_TO_VALUE']).otherwise(None))

In [32]:
# Replace the missing values with the median for training

df = df.withColumn('ORIGINAL_COMBINED_LOAN_TO_VALUE', when(df['ORIGINAL_COMBINED_LOAN_TO_VALUE'].isNotNull(), df['ORIGINAL_COMBINED_LOAN_TO_VALUE']).otherwise(80))

### Explore ORIGINAL_LOAN_TO_VALUE of the property

In [33]:
# An original LTV is unavailable when its value is 999.

print('The number of loan records with unavailable original LTVs:', df.filter(df['ORIGINAL_LOAN_TO_VALUE'] == 999).count())

The number of loan records with unavailable original LTVs: 1


In [34]:
# Replace unavailable values with None.

df = df.withColumn('ORIGINAL_LOAN_TO_VALUE', when(df['ORIGINAL_LOAN_TO_VALUE'] != 999, df['ORIGINAL_LOAN_TO_VALUE']).otherwise(None))

In [35]:
# Replace  the missing values with the median.

df = df.withColumn('ORIGINAL_LOAN_TO_VALUE', when(df['ORIGINAL_LOAN_TO_VALUE'].isNotNull(), df['ORIGINAL_LOAN_TO_VALUE']).otherwise(80))

In [36]:
# Replace mIssing with mean of 327
df = df.withColumn('ORIGINAL_LOAN_TERM', when(df['ORIGINAL_LOAN_TERM'].isNotNull(), df['ORIGINAL_LOAN_TERM']).otherwise(327))

In [37]:
df.printSchema()

root
 |-- CREDIT_SCORE: double (nullable = true)
 |-- FIRST_TIME_HOMEBUYER_FLAG: string (nullable = true)
 |-- METROPOLITAN_STATISTICAL_AREA: double (nullable = true)
 |-- NUMBER_OF_UNITS: double (nullable = true)
 |-- OCCUPANCY_STATUS: string (nullable = true)
 |-- ORIGINAL_COMBINED_LOAN_TO_VALUE: double (nullable = true)
 |-- ORIGINAL_LOAN_TO_VALUE: double (nullable = true)
 |-- CHANNEL: string (nullable = true)
 |-- PROPERTY_TYPE: string (nullable = true)
 |-- POSTAL_CODE: double (nullable = true)
 |-- LOAN_PURPOSE: string (nullable = true)
 |-- ORIGINAL_LOAN_TERM: double (nullable = true)
 |-- NUMBER_OF_BORROWERS: double (nullable = true)
 |-- DELINQUENT: long (nullable = true)



Check amount of Denlinquents and Non Delinquents, OverSampling

In [38]:
df.count()

149997

In [39]:
#stop

In [40]:
# Split the dataframe into positive (DELINQUENT = 1) and negative (DELINQUENT = 0) examples
positive_df = df.filter(col("DELINQUENT") == 1)
negative_df = df.filter(col("DELINQUENT") == 0)

In [41]:
print(positive_df.count())
print(negative_df.count())

14830
135167


In [42]:
# Calculate the ratio of positive to negative examples
positive_count = positive_df.count()
negative_count = negative_df.count()
oversample_ratio = negative_count / positive_count
print(oversample_ratio)
# there is a high ratio of positive to negative, but for this scenario 2 should be enough
oversample_ratio = 2.0

9.114430209035739


In [43]:
# Oversample the positive examples
oversampled_positive_df = positive_df.sample(withReplacement=True, fraction=oversample_ratio, seed=42)

# Combine the oversampled positive examples with the original negative examples
balanced_df = negative_df.unionAll(oversampled_positive_df)


In [44]:
oversampled_positive_df.count()

29516

In [45]:
balanced_df.count()

164683

In [46]:
balanced_df.printSchema()

root
 |-- CREDIT_SCORE: double (nullable = true)
 |-- FIRST_TIME_HOMEBUYER_FLAG: string (nullable = true)
 |-- METROPOLITAN_STATISTICAL_AREA: double (nullable = true)
 |-- NUMBER_OF_UNITS: double (nullable = true)
 |-- OCCUPANCY_STATUS: string (nullable = true)
 |-- ORIGINAL_COMBINED_LOAN_TO_VALUE: double (nullable = true)
 |-- ORIGINAL_LOAN_TO_VALUE: double (nullable = true)
 |-- CHANNEL: string (nullable = true)
 |-- PROPERTY_TYPE: string (nullable = true)
 |-- POSTAL_CODE: double (nullable = true)
 |-- LOAN_PURPOSE: string (nullable = true)
 |-- ORIGINAL_LOAN_TERM: double (nullable = true)
 |-- NUMBER_OF_BORROWERS: double (nullable = true)
 |-- DELINQUENT: long (nullable = true)



## Train a Gradient Boosting Decision Tree Model

In [47]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [48]:
# make one-hot encoding for string categorical features
categorical_features = [
    'FIRST_TIME_HOMEBUYER_FLAG',
    'LOAN_PURPOSE',
    'NUMBER_OF_UNITS',
    'OCCUPANCY_STATUS',
    'CHANNEL',
    'PROPERTY_TYPE'
]

out_categorical_index_features = ['INDEX_FIRST_TIME_HOMEBUYER_FLAG',
                                  'INDEX_LOAN_PURPOSE',
                                  'INDEX_NUMBER_OF_UNITS',
                                  'INDEX_OCCUPANCY_STATUS',
                                  'INDEX_CHANNEL',
                                  'INDEX_PROPERTY_TYPE'
                                  ]

out_categorical_onehot_features = ['FIRST_TIME_HOMEBUYER_FLAG_VEC',
                                   'LOAN_PURPOSE_VEC',
                                   'NUMBER_OF_UNITS_VEC',
                                   'OCCUPANCY_STATUS_VEC',
                                   'CHANNEL_VEC',
                                   'PROPERTY_TYPE_VEC'
                                   ]


In [49]:
# Change Categorical Values into Numeric Values by changing into String Index then OneHot

string_encoder = StringIndexer()
string_encoder.setInputCols(categorical_features)
string_encoder.setOutputCols(out_categorical_index_features)
indexed_df = string_encoder.fit(balanced_df).transform(balanced_df)
indexed_df = indexed_df.drop(*categorical_features)

onehot_encoder = OneHotEncoder()
onehot_encoder.setInputCols(out_categorical_index_features)
onehot_encoder.setOutputCols(out_categorical_onehot_features)
encoded_df = onehot_encoder.fit(indexed_df).transform(indexed_df)
encoded_df = encoded_df.drop(*out_categorical_index_features)



In [50]:
encoded_df.printSchema()

root
 |-- CREDIT_SCORE: double (nullable = true)
 |-- METROPOLITAN_STATISTICAL_AREA: double (nullable = true)
 |-- ORIGINAL_COMBINED_LOAN_TO_VALUE: double (nullable = true)
 |-- ORIGINAL_LOAN_TO_VALUE: double (nullable = true)
 |-- POSTAL_CODE: double (nullable = true)
 |-- ORIGINAL_LOAN_TERM: double (nullable = true)
 |-- NUMBER_OF_BORROWERS: double (nullable = true)
 |-- DELINQUENT: long (nullable = true)
 |-- FIRST_TIME_HOMEBUYER_FLAG_VEC: vector (nullable = true)
 |-- LOAN_PURPOSE_VEC: vector (nullable = true)
 |-- NUMBER_OF_UNITS_VEC: vector (nullable = true)
 |-- OCCUPANCY_STATUS_VEC: vector (nullable = true)
 |-- CHANNEL_VEC: vector (nullable = true)
 |-- PROPERTY_TYPE_VEC: vector (nullable = true)



In [51]:
pandas_df = balanced_df.toPandas()

In [52]:
shape = pandas_df.shape

print("shape:", shape)

shape: (164683, 14)


In [53]:
schema = pandas_df.dtypes
print(schema)

CREDIT_SCORE                       float64
FIRST_TIME_HOMEBUYER_FLAG           object
METROPOLITAN_STATISTICAL_AREA      float64
NUMBER_OF_UNITS                    float64
OCCUPANCY_STATUS                    object
ORIGINAL_COMBINED_LOAN_TO_VALUE    float64
ORIGINAL_LOAN_TO_VALUE             float64
CHANNEL                             object
PROPERTY_TYPE                       object
POSTAL_CODE                        float64
LOAN_PURPOSE                        object
ORIGINAL_LOAN_TERM                 float64
NUMBER_OF_BORROWERS                float64
DELINQUENT                           int64
dtype: object


In [54]:
null_columns = pandas_df.columns[pandas_df.isnull().any()]
if null_columns.empty:
    print("No null values found in any column.")
else:
    print("Null values found in the following columns:")
    for col in null_columns:
        print(col)

No null values found in any column.


In [55]:
# Label Encode
label_encoder = LabelEncoder()
categorical_cols = ["FIRST_TIME_HOMEBUYER_FLAG", "OCCUPANCY_STATUS", "CHANNEL", "PROPERTY_TYPE", "LOAN_PURPOSE"]
for col in categorical_cols:
    pandas_df[col] = label_encoder.fit_transform(pandas_df[col])

In [96]:
# Save the label encoder

import pickle

output = open('encoderModel.pkl', 'wb')
pickle.dump(label_encoder, output)
output.close()

In [97]:
# pkl_file = open('encoderModel.pkl', 'rb')
# label_encoder = pickle.load(pkl_file) 
# pkl_file.close()

In [56]:
pandas_df.head()

,CREDIT_SCORE,FIRST_TIME_HOMEBUYER_FLAG,METROPOLITAN_STATISTICAL_AREA,NUMBER_OF_UNITS,OCCUPANCY_STATUS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_LOAN_TO_VALUE,CHANNEL,PROPERTY_TYPE,POSTAL_CODE,LOAN_PURPOSE,ORIGINAL_LOAN_TERM,NUMBER_OF_BORROWERS,DELINQUENT
0,686.0,0,0.0,1.0,1,80.0,80.0,2,4,49400.0,0,360.0,1.0,0
1,692.0,0,19740.0,1.0,1,79.0,79.0,2,3,80100.0,0,360.0,2.0,0
2,787.0,0,48424.0,1.0,1,75.0,75.0,2,3,33400.0,0,180.0,2.0,0
3,744.0,1,0.0,1.0,1,97.0,97.0,2,4,50400.0,2,360.0,1.0,0
4,790.0,0,28940.0,1.0,1,70.0,70.0,2,4,37800.0,1,180.0,1.0,0


In [57]:
X = pandas_df.drop("DELINQUENT", axis=1)
y = pandas_df["DELINQUENT"]


In [58]:
X.shape

(164683, 13)

In [59]:
y.shape

(164683,)

In [60]:
schema = X.dtypes
print(schema)


CREDIT_SCORE                       float64
FIRST_TIME_HOMEBUYER_FLAG            int32
METROPOLITAN_STATISTICAL_AREA      float64
NUMBER_OF_UNITS                    float64
OCCUPANCY_STATUS                     int32
ORIGINAL_COMBINED_LOAN_TO_VALUE    float64
ORIGINAL_LOAN_TO_VALUE             float64
CHANNEL                              int32
PROPERTY_TYPE                        int32
POSTAL_CODE                        float64
LOAN_PURPOSE                         int32
ORIGINAL_LOAN_TERM                 float64
NUMBER_OF_BORROWERS                float64
dtype: object


In [61]:
X.head()

,CREDIT_SCORE,FIRST_TIME_HOMEBUYER_FLAG,METROPOLITAN_STATISTICAL_AREA,NUMBER_OF_UNITS,OCCUPANCY_STATUS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_LOAN_TO_VALUE,CHANNEL,PROPERTY_TYPE,POSTAL_CODE,LOAN_PURPOSE,ORIGINAL_LOAN_TERM,NUMBER_OF_BORROWERS
0,686.0,0,0.0,1.0,1,80.0,80.0,2,4,49400.0,0,360.0,1.0
1,692.0,0,19740.0,1.0,1,79.0,79.0,2,3,80100.0,0,360.0,2.0
2,787.0,0,48424.0,1.0,1,75.0,75.0,2,3,33400.0,0,180.0,2.0
3,744.0,1,0.0,1.0,1,97.0,97.0,2,4,50400.0,2,360.0,1.0
4,790.0,0,28940.0,1.0,1,70.0,70.0,2,4,37800.0,1,180.0,1.0


In [62]:
schema = y.dtypes
print(schema)

int64


In [63]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: DELINQUENT, dtype: int64

In [64]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [65]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(131746, 13)
(32937, 13)
(131746,)
(32937,)


In [66]:
best_params = {'colsample_bytree': 1.0,
 'learning_rate': 0.1,
 'max_depth': 15,
 'n_estimators': 400,
 'subsample': 0.8}

In [67]:
#STOP
# 

XGBClassifier With Best Params



In [68]:
start_time = datetime.now()
#clf = xgb.XGBClassifier(use_label_encoder=False, n_estimators=10)
clf = xgb.XGBClassifier(**best_params)
clf.fit(X_train, y_train.astype(int))
end_time = datetime.now()
training_time = end_time - start_time

preds = clf.predict(X_test)

print("Training time = %s" % training_time)


Training time = 0:00:11.136069


In [69]:

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)

# Create confusion matrix
confusion = confusion_matrix(y_test, preds)
confusion_df = pd.DataFrame(confusion, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1'])

# Print evaluation metrics and confusion matrix
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion_df)

Accuracy: 0.8937365273097125
Precision: 0.8699064292182312
Recall: 0.48428835489833644
F1 Score: 0.6221934369602763
Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0        26555          431
Actual 1         3069         2882


In [70]:
# Too Big Plot Decision Tree
# plot_tree(clf, rankdir='LR', num_trees=1)
# plt.show()

In [71]:
# Get feature importance
importance = clf.feature_importances_

# Create a DataFrame to display feature importance
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print or display the feature importance
print(feature_importance_df)

                            Feature  Importance
12              NUMBER_OF_BORROWERS    0.090928
0                      CREDIT_SCORE    0.089955
6            ORIGINAL_LOAN_TO_VALUE    0.084589
3                   NUMBER_OF_UNITS    0.083972
4                  OCCUPANCY_STATUS    0.082405
2     METROPOLITAN_STATISTICAL_AREA    0.076933
9                       POSTAL_CODE    0.076931
11               ORIGINAL_LOAN_TERM    0.075346
8                     PROPERTY_TYPE    0.070059
1         FIRST_TIME_HOMEBUYER_FLAG    0.068128
5   ORIGINAL_COMBINED_LOAN_TO_VALUE    0.068068
7                           CHANNEL    0.066859
10                     LOAN_PURPOSE    0.065827


In [ ]:
# Save Model

In [72]:
#stop

### Testing

In [73]:
# Sample data
data = [
    [450, 'N', 12345.0, 1, 'I', 74,10, 'R', 'SF', 54321.0, 'P', 360.0, 1.0],
    [200, 'Y', 23456.0, 2, 'P', 115, 100, 'C', 'CO', 65432.0, 'N', 240.0, 2.0],
    [650, 'N', 34567.0, 1, 'S', 100, 30, 'B', 'CP', 76543.0, 'C', 180.0, 1.0],
    [500, 'Y', 45678.0, 2, 'I', 50, 40, 'B', 'MH', 87654.0, 'C', 300.0, 3.0],
    [500, 'Y', 47664.0, 2, 'P', 69, 69, 'C', 'SF', 48300.0, 'N', 240.0, 1.0],
]


In [74]:
# Column names based on the given schema
columns = [
    'CREDIT_SCORE',
    'FIRST_TIME_HOMEBUYER_FLAG',
    'METROPOLITAN_STATISTICAL_AREA',
    'NUMBER_OF_UNITS', 
    'OCCUPANCY_STATUS',
    'ORIGINAL_COMBINED_LOAN_TO_VALUE',
    'ORIGINAL_LOAN_TO_VALUE',
    'CHANNEL',
    'PROPERTY_TYPE',
    'POSTAL_CODE',
    'LOAN_PURPOSE',
    'ORIGINAL_LOAN_TERM',
    'NUMBER_OF_BORROWERS',
    
]

In [75]:
# Create the DataFrame
test_DF = pd.DataFrame(data, columns=columns)

In [76]:
# Label Encode

categorical_cols = ["FIRST_TIME_HOMEBUYER_FLAG", "OCCUPANCY_STATUS", "CHANNEL", "PROPERTY_TYPE", "LOAN_PURPOSE"]
for col in categorical_cols:
    test_DF[col] = label_encoder.fit_transform(test_DF[col])

In [77]:
preds = clf.predict_proba(test_DF)

In [78]:
preds

array([[0.9599252 , 0.04007483],
       [0.8585426 , 0.14145738],
       [0.9185379 , 0.08146206],
       [0.7496332 , 0.2503668 ],
       [0.9012769 , 0.09872311]], dtype=float32)

In [79]:
delinquent_entries = balanced_df[balanced_df['DELINQUENT'] == 1]

In [80]:
delinquent_entries.head(5)

# 

[Row(CREDIT_SCORE=755.0, FIRST_TIME_HOMEBUYER_FLAG='N', METROPOLITAN_STATISTICAL_AREA=47664.0, NUMBER_OF_UNITS=1.0, OCCUPANCY_STATUS='P', ORIGINAL_COMBINED_LOAN_TO_VALUE=69.0, ORIGINAL_LOAN_TO_VALUE=69.0, CHANNEL='C', PROPERTY_TYPE='SF', POSTAL_CODE=48300.0, LOAN_PURPOSE='N', ORIGINAL_LOAN_TERM=240.0, NUMBER_OF_BORROWERS=1.0, DELINQUENT=1),
 Row(CREDIT_SCORE=635.0, FIRST_TIME_HOMEBUYER_FLAG='N', METROPOLITAN_STATISTICAL_AREA=19124.0, NUMBER_OF_UNITS=1.0, OCCUPANCY_STATUS='P', ORIGINAL_COMBINED_LOAN_TO_VALUE=63.0, ORIGINAL_LOAN_TO_VALUE=63.0, CHANNEL='R', PROPERTY_TYPE='PU', POSTAL_CODE=75000.0, LOAN_PURPOSE='C', ORIGINAL_LOAN_TERM=360.0, NUMBER_OF_BORROWERS=1.0, DELINQUENT=1),
 Row(CREDIT_SCORE=752.0, FIRST_TIME_HOMEBUYER_FLAG='N', METROPOLITAN_STATISTICAL_AREA=0.0, NUMBER_OF_UNITS=1.0, OCCUPANCY_STATUS='P', ORIGINAL_COMBINED_LOAN_TO_VALUE=65.0, ORIGINAL_LOAN_TO_VALUE=65.0, CHANNEL='R', PROPERTY_TYPE='MH', POSTAL_CODE=27500.0, LOAN_PURPOSE='C', ORIGINAL_LOAN_TERM=240.0, NUMBER_OF_BORRO

In [81]:
#Stop

## Grid Search


In [82]:
param_grid = {
    'max_depth': [3, 5, 7,9,11,13,15],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100,150, 200,250, 300,350,400],
    'subsample': [0.8, 1.0,1.2,1.4],
    'colsample_bytree': [0.8, 1.0,1.2,1.4]
}

In [83]:
start_time = datetime.now()
model = xgb.XGBClassifier()
grid_search = GridSearchCV(model, param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)
end_time = datetime.now()
training_time = end_time - start_time
print("Training time = %s" % training_time)

C:\Users\kevin\anaconda3\envs\Capstone\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
8820 fits failed out of a total of 11760.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1470 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\kevin\anaconda3\envs\Capstone\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kevin\anaconda3\envs\Capstone\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "C:\Users\kevin\anaconda3\envs\Capstone\lib\site-packages\xgboost\sklearn.py", line 1490, in fit
    self._Booster = train(
  File "C:\Use

Training time = 5:21:56.386448


In [84]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_


In [85]:
best_params

{'colsample_bytree': 1.0,
 'learning_rate': 0.1,
 'max_depth': 15,
 'n_estimators': 400,
 'subsample': 0.8}

In [86]:
preds = best_model.predict(X_test)


In [87]:

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)

# Create confusion matrix
confusion = confusion_matrix(y_test, preds)
confusion_df = pd.DataFrame(confusion, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1'])

# Print evaluation metrics and confusion matrix
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion_df)

Accuracy: 0.9416461729969335
Precision: 0.8885245901639345
Recall: 0.7741556041001513
F1 Score: 0.8274066091954024
Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0        26408          578
Actual 1         1344         4607


In [88]:
# Get feature importance
importance = best_model.feature_importances_

# Create a DataFrame to display feature importance
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print or display the feature importance
print(feature_importance_df)

                            Feature  Importance
3                   NUMBER_OF_UNITS    0.095821
4                  OCCUPANCY_STATUS    0.093118
11               ORIGINAL_LOAN_TERM    0.082121
2     METROPOLITAN_STATISTICAL_AREA    0.079057
0                      CREDIT_SCORE    0.078646
6            ORIGINAL_LOAN_TO_VALUE    0.076779
9                       POSTAL_CODE    0.076368
8                     PROPERTY_TYPE    0.072075
5   ORIGINAL_COMBINED_LOAN_TO_VALUE    0.071214
12              NUMBER_OF_BORROWERS    0.070549
1         FIRST_TIME_HOMEBUYER_FLAG    0.070050
7                           CHANNEL    0.067176
10                     LOAN_PURPOSE    0.067027


In [89]:
best_model.save_model("model.json")

In [90]:
model_xgb_2 = xgb.XGBClassifier()
model_xgb_2.load_model("model.json")

In [91]:
model_xgb_2

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              feature_types=['float', 'int', 'float', 'float', 'int', 'float',
                             'float', 'int', 'int', 'float', 'int', 'float',
                             'float'],
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [92]:
preds3 = model_xgb_2.predict(X_test)

In [93]:
X_test.shape

(32937, 13)

In [94]:
preds3

array([0, 0, 0, ..., 0, 0, 0])

In [95]:
unique_values = np.unique(preds3)
count_unique = len(unique_values)

print("Unique values:", unique_values)
print("Count of unique values:", count_unique)

Unique values: [0 1]
Count of unique values: 2
